<html>
    <h1> Bagging and Random Forests
        </h1>
        <p> <b> Min Khant Soe</b> </p>
        <p> ID: 122277 </p>
        <p> ^_^ </p>
    </html>

<html>
    <h3> Modify the Bagging scratch code in our lecture such that:
        </h3>
        <p> Calculate for oob evaluation for each bootstrapped dataset, and also the average score</p>
        <p> Change the code to "without replacement"</p>
        <p> Put everything into a class Bagging. It should have at least two methods, fit(X_train, y_train), and predict(X_test)</p>
        <p> Modify the code from above to randomize features. Set the number of features to be used in each tree to be sqrt(n), and then select a subset of features for each tree. This can be easily done by setting our DecisionTreeClassifier max_features to 'sqrt'</p>
    </html>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.tree import DecisionTreeClassifier
import random
from scipy import stats
from sklearn.metrics import classification_report

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

from scipy import stats
from sklearn.metrics import classification_report, accuracy_score

In [2]:
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                test_size=0.4, shuffle=True, random_state=35)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(90, 4) (60, 4) (90,) (60,)


In [3]:
class Bagging:
    def set_params(self, B, bootstrap_ratio, with_no_replacement=True):
        self.B = B
        self.bootstrap_ratio = bootstrap_ratio
        self.with_no_replacement = with_no_replacement
              
    def fit(self, X, y): 
        m, n = X.shape
        
        tree_params = {'max_depth': 2, 'max_features': 'sqrt'}
        self.models = [DecisionTreeClassifier(**tree_params) for _ in range(self.B)]

        #sample size for each tree
        sample_size = int(self.bootstrap_ratio * len(X))

        xsamples = np.zeros((self.B, sample_size, n))
        ysamples = np.zeros((self.B, sample_size))

        xsamples_oob = []  #use list because length is not known
        ysamples_oob = []

        #bootstrapping samples for each model
        for i in range(self.B):
            oob_idx = []
            idxes = []
            
            for j in range(sample_size):
                idx = random.randrange(m)
                if (self.with_no_replacement):
                    while idx in idxes:
                        idx = random.randrange(m)
                idxes.append(idx)
                oob_idx.append(idx)
                xsamples[i, j, :] = X[idx]
                ysamples[i, j] = y[idx]
                
            mask = np.zeros((m), dtype=bool)
            mask[oob_idx] = True
            xsamples_oob.append(X[~mask])
            ysamples_oob.append(y[~mask])
    
        #fitting each estimator
        oob_score = 0
        print("Out of bag score for each tree")
        for i, model in enumerate(self.models):
            
            _X = xsamples[i]
            _y = ysamples[i]
            model.fit(_X, _y)

            #calculating oob score
            _X_test = np.asarray(xsamples_oob[i])
            _y_test = np.asarray(ysamples_oob[i])
            yhat = model.predict(_X_test)
            
            oob_score += accuracy_score(_y_test, yhat)
            accuracy = str(round(accuracy_score(_y_test, yhat), 4))
            print(f"Tree {i}", accuracy)
            
        self.avg_oob_score = str(round(oob_score / len(self.models), 4))
        print("\nAverage out of bag score")
        print(self.avg_oob_score)
    
    def predict(self, X): 
        #make prediction and return the probabilities
        predictions = np.zeros((self.B, X.shape[0]))
        for i, model in enumerate(self.models):
            yhat = model.predict(X)
            predictions[i, :] = yhat
        
        return stats.mode(predictions)[0][0]

In [4]:
model = Bagging()
model.set_params(B=10, bootstrap_ratio=0.9)
model.fit(X_train, y_train)
yhat = model.predict(X_test)
print('\n',classification_report(y_test, yhat))

Out of bag score for each tree
Tree 0 1.0
Tree 1 1.0
Tree 2 0.8889
Tree 3 1.0
Tree 4 0.4444
Tree 5 1.0
Tree 6 0.8889
Tree 7 1.0
Tree 8 1.0
Tree 9 1.0

Average out of bag score
0.9222

               precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       0.86      0.86      0.86        22
           2       0.84      0.84      0.84        19

    accuracy                           0.90        60
   macro avg       0.90      0.90      0.90        60
weighted avg       0.90      0.90      0.90        60

